
# Lab 8B — ChatGPT Study Assistant with Sidebar + Streaming

## Requirements
- **Python** 3.9+ (3.10 recommended)
- **Packages:** `streamlit`, `openai`
- **OpenAI API key** set in your environment as `OPENAI_API_KEY`
  - macOS/Linux: `export OPENAI_API_KEY="sk-..."`
  - Windows (PowerShell): `setx OPENAI_API_KEY "sk-..."` then restart the terminal

## Learning Objectives
- Wire a Streamlit chat app to OpenAI's Chat Completions API.
- Add a sidebar with model and temperature controls.
- Stream tokens for responsive UX.
- Provide helpful study tools (summarize, outline, quiz ideas).

---

### What you will build
A functional **Study Assistant** that accepts questions, can summarize or outline topics, suggests quiz questions, and streams responses while updating chat history.


In [ ]:

# If running locally and packages are missing, uncomment:
# !pip install -U streamlit openai

print("Ready. Next cell will write a runnable Streamlit app file for this lab.")


In [ ]:
from pathlib import Path
APP_PATH = Path("/mnt/data/lesson8_labs/app_lab8b.py")
APP_CODE = "# app_lab8b.py\nimport os, streamlit as st\ntry:\n    from openai import OpenAI\nexcept Exception:\n    OpenAI = None\n\nst.set_page_config(page_title=\"Lab 8B \u2014 Study Assistant\", page_icon=\"\ud83d\udcda\", layout=\"wide\")\nst.title(\"\ud83d\udcda Lab 8B \u2014 ChatGPT Study Assistant\")\n\n# -------- Sidebar settings --------\nwith st.sidebar:\n    st.header(\"\u2699\ufe0f Settings\")\n    model = st.selectbox(\"Model\", [\"gpt-4o-mini\", \"gpt-4o\", \"gpt-4.1-mini\"], index=0)\n    temperature = st.slider(\"Temperature\", 0.0, 1.0, 0.3, 0.05)\n    system_prompt = st.text_area(\"System Prompt\", \"You are a helpful, concise study assistant that explains clearly.\")\n\n# -------- API setup --------\napi_key = os.getenv(\"OPENAI_API_KEY\")\nif not api_key:\n    st.error(\"OPENAI_API_KEY not set. Please set it and restart the app.\")\n    st.stop()\n\nif OpenAI is None:\n    st.error(\"OpenAI SDK not installed. Run: pip install openai\")\n    st.stop()\n\nclient = OpenAI(api_key=api_key)\n\n# -------- Session state --------\nif \"messages\" not in st.session_state:\n    st.session_state.messages = []\n\n# -------- Input + Study tools --------\ncol_input, col_tools = st.columns([3, 1])\nwith col_input:\n    user_msg = st.text_area(\"Your message\", height=140, placeholder=\"e.g., 'Summarize the causes of World War I.'\")\n\nwith col_tools:\n    st.subheader(\"Study Tools\")\n    do_summarize = st.button(\"Summarize\")\n    do_outline = st.button(\"Outline\")\n    do_quiz = st.button(\"Quiz Ideas\")\n\ndef ask_openai(messages, stream=False):\n    if not stream:\n        chat = client.chat.completions.create(\n            model=model, messages=messages, temperature=temperature\n        )\n        return chat.choices[0].message.content\n    else:\n        stream_resp = client.chat.completions.create(\n            model=model, messages=messages, temperature=temperature, stream=True\n        )\n        return stream_resp\n\ndef respond(user_text, tool=None, use_stream=True):\n    if not user_text.strip():\n        return\n\n    messages = [{\"role\": \"system\", \"content\": system_prompt}]\n\n    # Short context window from prior messages (optional)\n    if st.session_state.messages:\n        prior = \"\\n\".join([f\"{m['role']}: {m['content']}\" for m in st.session_state.messages[-6:]])\n        messages.append({\"role\": \"system\", \"content\": f\"Recent context:\\n{prior}\"})\n\n    # Tool modifiers\n    if tool == \"summarize\":\n        user_text = f\"Summarize this topic in 5 concise bullets with definitions if needed:\\n\\n{user_text}\"\n    elif tool == \"outline\":\n        user_text = f\"Create a clear study outline with sections and bullet points for:\\n\\n{user_text}\"\n    elif tool == \"quiz\":\n        user_text = f\"Propose 5 practice questions (mix of MCQ and short-answer) with brief answers on:\\n\\n{user_text}\"\n\n    messages.append({\"role\": \"user\", \"content\": user_text.strip()})\n\n    if use_stream:\n        placeholder = st.empty()\n        output = \"\"\n        with st.spinner(\"Streaming response...\"):\n            try:\n                for event in ask_openai(messages, stream=True):\n                    delta = event.choices[0].delta\n                    if delta and delta.content:\n                        output += delta.content\n                        placeholder.markdown(output)\n            except Exception as e:\n                st.error(f\"Streaming error: {e}\")\n                return\n        st.session_state.messages.append({\"role\": \"user\", \"content\": user_text.strip()})\n        st.session_state.messages.append({\"role\": \"assistant\", \"content\": output})\n    else:\n        with st.spinner(\"Thinking...\"):\n            try:\n                reply = ask_openai(messages, stream=False)\n            except Exception as e:\n                st.error(f\"API error: {e}\")\n                return\n        st.session_state.messages.append({\"role\": \"user\", \"content\": user_text.strip()})\n        st.session_state.messages.append({\"role\": \"assistant\", \"content\": reply})\n\n# Buttons\nif st.button(\"Send\"):\n    respond(user_msg, tool=None, use_stream=True)\nif do_summarize:\n    respond(user_msg, tool=\"summarize\", use_stream=True)\nif do_outline:\n    respond(user_msg, tool=\"outline\", use_stream=True)\nif do_quiz:\n    respond(user_msg, tool=\"quiz\", use_stream=True)\n\nst.write('---')\nst.subheader(\"Conversation\")\nfor m in st.session_state.messages:\n    role = \"\ud83e\uddd1\u200d\ud83c\udf93 You\" if m[\"role\"] == \"user\" else \"\ud83e\udd16 Assistant\"\n    st.markdown(f\"**{role}**\")\n    st.markdown(m[\"content\"])\n\nif st.button(\"Clear Conversation\"):\n    st.session_state.messages = []\n    st.experimental_rerun()\n"
APP_PATH.write_text(APP_CODE, encoding='utf-8')
print(f'Wrote: {APP_PATH}')



### Run the app
In a terminal where the file was written (shown above):
```
streamlit run app_lab8b.py
```
**Tasks**
1. Ask the assistant a study question and observe a streamed response.
2. Use **Summarize**, **Outline**, and **Quiz Ideas** on the same topic and compare results.
3. Tweak **temperature** in the sidebar to see how style changes.
4. (Optional) Replace the model with a different one available to your account.

**Troubleshooting**
- If you see `OPENAI_API_KEY not set`, export your key and restart the terminal.
- If you see `OpenAI SDK not installed`, install it with `pip install openai`.



---
## Summary
- You built a functional ChatGPT-powered Study Assistant in Streamlit.
- You added a sidebar for model/temperature, and you implemented **streaming** for responsive UX.
- You practiced designing helpful study workflows (summarize/outline/quiz).
- You’re now ready to publish to Streamlit Community Cloud or extend with file uploads, citations, or RAG.
